In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import gudhi as gd
from gudhi.representations import PersistenceImage, BettiCurve, Entropy

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score, recall_score, balanced_accuracy_score

from scipy.ndimage import gaussian_filter
from scipy.stats import gaussian_kde

from pathlib import Path
from tqdm.notebook import tqdm

In [2]:
data_input = Path('data_segmented')
input_airyscan = data_input / 'Airyscan'
input_sted = data_input / 'STED'

data_pers = Path('data_processed')
pers_sted = data_pers / 'persistence_sted'
pers_airyscan = data_pers / 'persistence_airyscan'
vectorization_sted = data_pers / 'vectorizations_sted'
vectorization_airyscan = data_pers / 'vectorizations_airyscan'
classification_path = data_pers / 'classification'

figures_path = Path('figures_plots')

In [26]:
preproc_choice = 'clip_minmax_gaussian2c_minmax'

class_files = sorted([file for file in classification_path.rglob('*classification*traintest_runs*.csv')
                      if preproc_choice in file.name and 'sted' in file.name.lower()
                      and 'new' in file.name.lower()])
df_class_sted = pd.concat([pd.read_csv(file) for file in tqdm(class_files)])
df_class_sted = df_class_sted.sort_values(by=['dim', 'type', 'run'])

class_files = sorted([file for file in classification_path.rglob('*classification*traintest_runs*.csv')
                      if preproc_choice in file.name and 'airyscan' in file.name.lower()
                      and 'new' in file.name.lower()])
df_class_airy = pd.concat([pd.read_csv(file) for file in tqdm(class_files)])
df_class_airy = df_class_airy.sort_values(by=['dim', 'type', 'run'])

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
df_class_sted.drop(columns=['hash_train', 'hash_test', 'prefix']).groupby(['dim', 'type'])\
    .agg(['mean', 'std']).reset_index()

dim  type accuracy_balanced           accuracy_train           precision  \
                          mean       std           mean       std      mean   
0    2   all          0.978926  0.013564       0.998810  0.003174  0.981902   
1    2    bc          0.981518  0.010624       0.998810  0.001997  0.982460   
2    2   ecc          0.929422  0.022356       0.984354  0.019222  0.936506   
3    2   ent          0.971047  0.016022       0.999320  0.001779  0.976010   
4    2    pi          0.962123  0.015862       0.999830  0.000761  0.968061   
5    2  stat          0.970379  0.019998       0.996429  0.005991  0.973168   
6    3   all          0.977710  0.013860       0.997959  0.004040  0.980795   
7    3    bc          0.971007  0.014171       0.999490  0.001664  0.974224   
8    3   ecc          0.954621  0.014428       0.992007  0.010940  0.961454   
9    3   ent          0.973222  0.013576       0.997279  0.009846  0.976037   
10   3    pi          0.949570  0.018817       0.975170  0.015335  0.957763   
11   3  stat          0.977433  0.013648       0.988776  0.007245  0.977814   

                recall            run           
         std      mean       std mean      std  
0   0.009389  0.981522  0.009561  9.5  5.91608  
1   0.009295  0.982065  0.009512  9.5  5.91608  
2   0.018460  0.935598  0.018600  9.5  5.91608  
3   0.012850  0.975543  0.013254  9.5  5.91608  
4   0.013238  0.967391  0.013658  9.5  5.91608  
5   0.016726  0.972554  0.017048  9.5  5.91608  
6   0.011627  0.980435  0.012011  9.5  5.91608  
7   0.012529  0.973913  0.012788  9.5  5.91608  
8   0.011476  0.960598  0.011680  9.5  5.91608  
9   0.013018  0.975272  0.013825  9.5  5.91608  
10  0.015297  0.956793  0.015114  9.5  5.91608  
11  0.012610  0.977174  0.013382  9.5  5.91608

In [27]:
df_class_airy.drop(columns=['hash_train', 'hash_test', 'prefix']).groupby(['dim', 'type'])\
    .agg(['mean', 'std']).reset_index()

dim  type accuracy_balanced           accuracy_train           precision  \
                          mean       std           mean       std      mean   
0    2   all          0.753731  0.086168       0.991964  0.025930  0.785724   
1    2    bc          0.744583  0.083364       0.977321  0.050562  0.778112   
2    2   ecc          0.664754  0.074495       0.966429  0.073617  0.713089   
3    2   ent          0.742860  0.083373       0.972321  0.053262  0.779665   
4    2    pi          0.719867  0.077431       0.985179  0.041165  0.759396   
5    2  stat          0.721780  0.061426       0.989464  0.027582  0.760999   
6    3   all          0.900114  0.058462       0.999286  0.003517  0.910208   
7    3    bc          0.869564  0.079342       0.990179  0.017835  0.886982   
8    3   ecc          0.810322  0.068614       0.970536  0.044426  0.833333   
9    3   ent          0.850777  0.061832       0.989107  0.026118  0.868721   
10   3    pi          0.826780  0.084536       0.999286  0.004337  0.847766   
11   3  stat          0.911951  0.051136       0.996429  0.007614  0.919079   

                recall             run             
         std      mean       std  mean        std  
0   0.072860  0.749714  0.073785  49.5  29.011492  
1   0.070524  0.746286  0.073129  49.5  29.011492  
2   0.063856  0.678000  0.072173  49.5  29.011492  
3   0.067215  0.757429  0.072246  49.5  29.011492  
4   0.066222  0.704286  0.069675  49.5  29.011492  
5   0.052687  0.721429  0.070733  49.5  29.011492  
6   0.048199  0.899143  0.054552  49.5  29.011492  
7   0.065561  0.871429  0.079215  49.5  29.011492  
8   0.055991  0.805714  0.068557  49.5  29.011492  
9   0.051640  0.846000  0.065215  49.5  29.011492  
10  0.069802  0.828286  0.077849  49.5  29.011492  
11  0.042960  0.908286  0.047757  49.5  29.011492

# Classification: plots in the paper

In [ ]:
# pick the best perfomer except stat or all

df_plot = df_class_sted.loc[(df_class_sted['dim'] == 3), :].copy()
df_plot = df_plot.drop(columns=['dim', 'prefix', 'hash_train', 'hash_test', 'run', 'accuracy_train'])
df_melt = df_plot.melt(id_vars=['type'], var_name='metric', value_name='value')

df_plot = df_melt.loc[(df_melt['type'] == 'bc') | (df_melt['type'] == 'stat') | (df_melt['type'] == 'pi'), :].copy()
df_plot.loc[:, 'type'] = df_plot['type'].replace({'bc': 'Betti curve', 'stat': 'Persistence Statistics', 'pi': 'Persistence Image'})
df_plot.loc[:, 'metric'] = df_plot['metric'].replace({'accuracy_balanced': 'Accuracy',
                             'precision': 'Precision',
                             'recall': 'Recall'})

df_plot_sted = df_plot.copy()
fig, ax = plt.subplots(figsize=(5, 6), dpi=300)
sns.boxplot(data=df_plot, x='type', y='value', hue='metric', ax=ax)
ax.set_xlabel('Vectorization method')
ax.set_ylabel('')
ax.set_title('STED')
fig.savefig(figures_path / 'sted_vectorization_performance_splitintometrics.png', dpi=300)

df_choice = df_melt.loc[df_melt['type']=='bc',:]
df_choice.loc[:, 'metric'] = df_choice['metric'].replace({'accuracy_balanced': 'Accuracy',
                             'precision': 'Precision',
                             'recall': 'Recall'})

In [ ]:
# pick the best perfomer except stat or all

df_plot = df_class_airy.loc[(df_class_airy['dim'] == 3), :].copy()
df_plot = df_plot.drop(columns=['dim', 'prefix', 'hash_train', 'hash_test', 'run', 'accuracy_train'])
df_melt = df_plot.melt(id_vars=['type'], var_name='metric', value_name='value')

df_plot = df_melt.loc[(df_melt['type'] == 'bc') | (df_melt['type'] == 'stat') | (df_melt['type'] == 'pi'), :].copy()
df_plot.loc[:, 'type'] = df_plot['type'].replace({'bc': 'Betti curve', 'stat': 'Persistence Statistics', 'pi': 'Persistence Image'})
df_plot.loc[:, 'metric'] = df_plot['metric'].replace({'accuracy_balanced': 'Accuracy',
                             'precision': 'Precision',
                             'recall': 'Recall'})

df_plot_airy = df_plot.copy()
fig, ax = plt.subplots(figsize=(5, 6), dpi=300)
sns.boxplot(data=df_plot, x='type', y='value', hue='metric', ax=ax)
ax.set_xlabel('Vectorization method')
ax.set_ylabel('')
ax.set_title('Airyscan')

df_choice2 = df_melt.loc[df_melt['type']=='bc',:]
df_choice2.loc[:, 'metric'] = df_choice2['metric'].replace({'accuracy_balanced': 'Accuracy',
                             'precision': 'Precision',
                             'recall': 'Recall'})
fig.savefig(figures_path / 'airyscan_vectorization_performance_splitintometrics.png', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(11, 6), dpi=300)
sns.boxplot(data=df_plot_sted, x='type', y='value', hue='metric', ax=ax[0])
ax[0].set_xlabel('Vectorization method')
ax[0].set_ylabel('')
ax[0].set_title('STED')

sns.boxplot(data=df_plot_airy, x='type', y='value', hue='metric', ax=ax[1])
ax[1].set_xlabel('Vectorization method')
ax[1].set_ylabel('')
ax[1].set_title('Airyscan')
fig.savefig(figures_path / 'airyscan-sted_vectorization_performance_splitintometrics.png', dpi=300)

In [ ]:
df_choice2.loc[:, 'type'] = 'Airyscan'
df_choice.loc[:, 'type'] = 'Sted'

df_choice_all = pd.concat([df_choice, df_choice2])

df_choice_all.groupby(['type', 'metric']).count()

In [ ]:
df_choice_all.groupby(['type', 'metric']).mean()

In [ ]:
# pick betti curves

fig, ax = plt.subplots(1,1, figsize=(3,10), dpi=400)
sns.boxplot(data=df_choice_all, x='type', y='value', hue='metric',
            ax=ax,
            width=1, 
            # notch=True, 
            showcaps=False,
            medianprops={"color": "black", "linewidth": 1},
            # positions=[0]
            # native_scale=True
            )
for i, spine in enumerate(ax.spines.values()):
    if i > 0:
        spine.set_visible(False)
ax.set_xticks([])
ax.set_xlabel("")
ax.tick_params(axis='x', which='both',length=0)

ax.set_ylabel('Score', fontsize=18)
ax.set_ylim(0.68,1.0)

# make the yticks smaller (in an iteration of 0.01)
# ax.yaxis.set_major_locator(plt.MultipleLocator(0.01))

# Modify legend: remove boxes and color labels
handles, labels = ax.get_legend_handles_labels()
# colors = [h.get_facecolor()[0][:3] for h in handles]  # Extract only RGB (ignore alpha)
colors = [tuple(list(h.get_facecolor())[:3]) for h in handles]

# Remove the original legend
ax.legend().remove()

# Create new legend with colored text
# for i, (label, color) in enumerate(zip(labels, colors)):
#     ax.text(0.55, 0.97 - i * 0.03, label, color=color, fontsize=20, transform=ax.transAxes)
for i, (label, color) in enumerate(zip(labels, colors)):
    ax.text(0.02, 0.65 - i * 0.05, label, color=color, fontsize=20, transform=ax.transAxes)

ax.text(0.55, 0.75, 'Airyscan', color='black', fontsize=18, transform=ax.transAxes, style='italic')
ax.text(0.1, 0.82, 'STED', color='black', fontsize=18, transform=ax.transAxes, style='italic')
# Change the text font to Arial
plt.rcParams['font.family'] = 'Arial'

fig.savefig(figures_path / 'boxplot_sted_vs_airy.png', bbox_inches='tight')
fig.savefig(figures_path / 'boxplot_sted_vs_airy.pdf', bbox_inches='tight')

# Dimensionality reduction: plots in the paper

## STED (persistence image or betti curve)

In [ ]:
df_tmp = pd.read_csv(Path(vectorization_sted, 'vectorizations_all_sted_clip_minmax_gaussian2c_minmax.csv'))
selection = [x for x in df_tmp.columns if 'labels' not in x and 'pi' in x and int(x.split('_')[2]) == 3]
Xt = df_tmp[selection].values
Xt = MinMaxScaler().fit_transform(Xt)

# Assume X and y are already defined
pca = PCA(n_components=3)
X = pca.fit_transform(Xt)
y = df_tmp['labels'].values.copy()

varexplained = np.sum(pca.explained_variance_ratio_)
print(varexplained)

from mpl_toolkits.mplot3d import Axes3D
from sklearn.svm import SVC

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

red = (1.0, 0.0, 0.0)
blue = (0.0, 0.0, 1.0)
colors = [red if yx == 0 or yx == 'ES' else blue for yx in y]

# Scatter plot with color based on y values
sc = ax.scatter(X[:, 0], X[:, 1], X[:, 2], 
                c=colors, marker='o')
ax.azim = 125
ax.elev = 15

# Train a linear SVM
svm = SVC(kernel='linear')
svm.fit(X, y)

# Get the separating hyperplane
w = svm.coef_[0]
a = -w[0] / w[2]
xx = np.linspace(min(X[:, 0]), max(X[:, 0]))
yy = np.linspace(min(X[:, 1]), max(X[:, 1]))
YY, XX = np.meshgrid(yy, xx)

# Compute ZZ only if w[2] is not zero
if abs(w[2]) > 1e-10:  # Prevent division by zero or near-zero errors
    ZZ = (-w[0] * XX - w[1] * YY - b) / w[2]
else:
    ZZ = np.full_like(XX, np.mean(X[:, 2]))  # Use mean Z-plane if w[2] is too small

# Plot the separating hyperplane
ax.plot_surface(XX, YY, ZZ, color='black', alpha=0.8)

In [ ]:
df_tmp = pd.read_csv(Path(vectorization_sted, 'vectorizations_all_sted_clip_minmax_gaussian2c_minmax.csv'))
selection = [x for x in df_tmp.columns if 'labels' not in x and 'pi' in x and int(x.split('_')[2]) == 3]
Xt = df_tmp[selection].values
Xt = MinMaxScaler().fit_transform(Xt)
y = df_tmp['labels'].copy()

# Assume X and y are already defined
pca = PCA(n_components=3)
X = pca.fit_transform(Xt)

print(np.sum(pca.explained_variance_ratio_))

svm = SVC(kernel='linear')
svm.fit(X, y)
print(svm.score(X, y))

# Get the separating hyperplane
w = svm.coef_[0]
a = -w[0] / w[2]

xx = np.linspace(min(X[:, 0]), max(X[:, 0]))
yy = np.linspace(min(X[:, 1]), max(X[:, 1]))
YY, XX = np.meshgrid(yy, xx)
# YY, XX = np.meshgrid(np.unique(X[:,1]), np.unique(X[:,0]))

# Plot the separating hyperplane
if abs(w[2]) > 1e-10:
    ZZ = (-w[0] * XX - w[1] * YY - svm.intercept_[0]) / w[2]
else:
    ZZ = np.full_like(XX, np.mean(X[:, 2]))

# ax.plot_surface(XX, YY, ZZ, color='black', alpha=0.9)
mask = (ZZ >= np.min(X[:,2])) & (ZZ <= np.max(X[:,2]))
XX, YY, ZZ = XX[mask], YY[mask], ZZ[mask]

fig = plt.figure(figsize=(8, 8), dpi=300)
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(XX.ravel(), YY.ravel(), ZZ.ravel(), color='black', alpha=0.4)

# plot the points
idx_right = np.where(svm.predict(X) == y)[0]
idx_wrong = np.where(svm.predict(X) != y)[0]

colors_right = np.array([red if yx == 0 or yx == 'ES' else blue for yx in y[idx_right]])
colors_wrong = np.array([red if yx == 0 or yx == 'ES' else blue for yx in y[idx_wrong]])

red = (1.0, 0.0, 0.0)
blue = (0.0, 0.0, 1.0)
colors = np.array([red if yx == 0 or yx == 'ES' else blue for yx in y])

# Scatter plot with color based on y values
sc = ax.scatter(X[idx_right, 0], X[idx_right, 1], X[idx_right, 2], 
                c=colors_right, marker='o', alpha=0.7)
sc = ax.scatter(X[idx_wrong, 0], X[idx_wrong, 1], X[idx_wrong, 2], 
                c=colors_wrong, marker='x', alpha=0.7)

ax.set_box_aspect([1,1,1])
ax.margins(0)

# ax.azim = 1.87
ax.azim = 10
ax.elev = 20
ax.dist = 1

In [ ]:
try:
    import plotly.graph_objects as go

except ImportError as e:
    print("Required module is not installed:", e)

# Create figure
fig = go.Figure()

# Scatter plot of data points
fig.add_trace(go.Scatter3d(
    x=X[idx_right, 0],
    y=X[idx_right, 1],
    z=X[idx_right, 2],
    mode='markers',
    marker=dict(color=colors[idx_right], size=5),
    showlegend=True
))
fig.add_trace(go.Scatter3d(
    x=X[idx_wrong, 0],
    y=X[idx_wrong, 1],
    z=X[idx_wrong, 2],
    mode='markers',
    marker=dict(color=colors[idx_wrong], size=3, symbol='x')
))
fig.update_layout(
    width=1000,  # Increase width
    height=800,  # Increase height
)

# Generate hyperplane grid
xx = np.linspace(min(X[:, 0]), max(X[:, 0]), 20)
yy = np.linspace(min(X[:, 1]), max(X[:, 1]), 20)
YY, XX = np.meshgrid(yy, xx)

if abs(w[2]) > 1e-6:
    ZZ = (-w[0] * XX - w[1] * YY - svm.intercept_[0]) / w[2]
else:
    ZZ = np.full_like(XX, np.mean(X[:, 2]))

# Add hyperplane as a surface
fig.add_trace(go.Surface(x=XX, y=YY, z=ZZ, colorscale='Greys', opacity=0.5, showscale=False))

# Update layout for interactivity
fig.update_layout(
    scene=dict(
        xaxis_title="PC1",
        yaxis_title="PC2",
        zaxis_title="PC3"
    ),
    title="Interactive 3D SVM Hyperplane",
    showlegend=False,
    scene_camera=dict(
        eye=dict(x=1.87, y=0.88, z=-0.64)
    )
)
# Extract camera settings from a Plotly figure
camera = fig.layout.scene.camera

azim = camera.eye.x  # Azimuth-like value
elev = camera.eye.y  # Elevation-like value
dist = camera.eye.z  # Distance from the plot

print(f"Azimuth: {azim}, Elevation: {elev}, Distance: {dist}")

fig.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import svm

df_tmp = pd.read_csv(Path(vectorization_sted, 'vectorizations_all_sted_clip_minmax_gaussian2c_minmax.csv'))
selection = [x for x in df_tmp.columns if 'labels' not in x and 'pi' in x and int(x.split('_')[2]) == 3]
Xt = df_tmp[selection].values
Xt = MinMaxScaler().fit_transform(Xt)
y = df_tmp['labels'].copy()

# Assume X and y are already defined
pca = PCA(n_components=3)
X = pca.fit_transform(Xt)

varexplained = np.sum(pca.explained_variance_ratio_)
print(varexplained)

# Train SVM with a linear kernel
model = svm.SVC(kernel='linear')
clf = model.fit(X, y)

# Extract hyperplane parameters
w = clf.coef_[0]  # Normal vector to the hyperplane
b = clf.intercept_[0]

# Compute two orthonormal basis vectors that span the hyperplane
w_norm = w / np.linalg.norm(w)  # Normalize w
U, _, _ = np.linalg.svd(np.eye(3) - np.outer(w_norm, w_norm))  # Find orthonormal basis
basis_1, basis_2 = U[:, 1], U[:, 2]  # Two vectors spanning the hyperplane

# Project points onto the hyperplane
distances = (X @ w + b) / np.linalg.norm(w)  # Signed distances
X_hyper = X - np.outer(distances, w_norm)  # Projected points

# Transform hyperplane points into 2D
X_2D = np.column_stack([X_hyper @ basis_1, X_hyper @ basis_2])
Xorg_2D = np.column_stack([X @ basis_1, X @ basis_2])

# Rotate to align with the diagonal (45-degree rotation)
theta = np.pi
rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
X_2D_rotated = X_2D @ rotation_matrix
Xorg_2D_rotated = Xorg_2D @ rotation_matrix

y_unq = np.unique(X_2D_rotated[:,1])
assert np.all(np.diff(y_unq) < 1e-14)
y_unq = np.mean(y_unq)


red = (1.0, 0.0, 0.0)
blue = (0.0, 0.0, 1.0)
colors = [red if yx == 0 or yx == 'ES' else blue for yx in y]

# Scatter plot in 2D
plt.figure(figsize=(8, 6), dpi=400)
plt.hlines(0, np.min(Xorg_2D_rotated[:, 0]), np.max(Xorg_2D_rotated[:, 0]), color='black', linestyle='--')
plt.scatter(Xorg_2D_rotated[:, 0], Xorg_2D_rotated[:, 1]-y_unq,
            color=colors, marker='x', label="Original Points")

# Labels
plt.xlabel("Projected Dimension 1")
plt.ylabel("Projected Dimension 2")
plt.title("2D Projection: Hyperplane Aligned to Diagonal")
plt.legend()
plt.grid()

In [ ]:
import matplotlib.lines as mlines

plt.rcParams['font.family'] = 'Arial'

# Scatter plot in 2D
fig, ax = plt.subplots(figsize=(10,10), dpi=400)
ax.hlines(0, np.min(X_2D_rotated[:, 0]), np.max(X_2D_rotated[:, 0]), color='black')

ypredict = model.predict(X)
idx_right = ypredict == y
idx_wrong = ypredict != y

# ax.scatter(Xorg_2D_rotated[idx_right, 0], Xorg_2D_rotated[idx_right, 1]-y_unq,
#            c=y[idx_right],
#            cmap="coolwarm",
#            marker='.',
#            label="Correct Points")
# ax.scatter(Xorg_2D_rotated[idx_wrong, 0], Xorg_2D_rotated[idx_wrong, 1]-y_unq,
#            c=y[idx_wrong],
#            cmap="coolwarm",
#            marker='x',
#            label="Incorrect Points")

red = (1.0, 0.0, 0.0)
blue = (0.0, 0.0, 1.0)
colors = [red if yx == 0 or yx == 'ES' else blue for yx in y]

idx_sort = np.argsort(Xorg_2D_rotated[:, 0])
Xplot = Xorg_2D_rotated[idx_sort, :]
yplot = y[idx_sort]
idx_right = idx_right[idx_sort]
idx_wrong = idx_wrong[idx_sort]

colors_right = [red if yx == 0 or yx == 'ES' else blue for yx in yplot[idx_right]]
colors_wrong = [red if yx == 0 or yx == 'ES' else blue for yx in yplot[idx_wrong]]

ax.scatter(Xplot[idx_right, 0], Xplot[idx_right, 1]-y_unq,
           c=colors_right,
           marker='.',
           s=120, #edgecolors='black', #alpha=0.8,
           label="Correct Points")
ax.scatter(Xplot[idx_wrong, 0], Xplot[idx_wrong, 1]-y_unq,
           c=colors_wrong,
           marker='x',
           s=100, #edgecolors='black',
           lw=2,
           label="Incorrect Points")

# we have two outliers; we remove them
sortedxvals = np.sort(Xorg_2D_rotated[:, 0])
lastx_incluster = sortedxvals[np.argmax(np.diff(sortedxvals))]

idx_inside = np.where(Xorg_2D_rotated[:, 0] < lastx_incluster)[0]
ax.set_xlim(sortedxvals[0]-0.5, lastx_incluster+0.5)
ax.set_ylim(np.min(Xorg_2D_rotated[idx_inside, 1])-y_unq-0.1,
            np.max(Xorg_2D_rotated[idx_inside, 1])-y_unq+0.1)

# Labels
plt.xlabel("projected dimension 1", fontsize=18)
plt.ylabel("projected dimension 2", fontsize=18)
plt.title("STED: 2D Projection of PCA of Persistence Image values\nAccuracy = {:.1f}% for {:d} samples".format(np.mean(idx_right)*100, len(idx_right)),
          fontsize=20)
# Add two points to the legend

handles, labels = ax.get_legend_handles_labels()
colors = plt.cm.coolwarm([0,300])

# handles.append(plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=red, markersize=6, label='ES'))
# handles.append(plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=blue, markersize=6, label='NS'))

handles = [mlines.Line2D([], [], marker='s', color='w', markerfacecolor=red, markersize=6, label='ES'),
           mlines.Line2D([], [], marker='s', color='w', markerfacecolor=blue, markersize=6, label='NS'),
           mlines.Line2D([], [], marker='x', color='black', markersize=6, linestyle=None, lw=0, label='wrong'),
           mlines.Line2D([], [], marker='o', color='w', markerfacecolor='black', markersize=6, label='correct')
           ]
ax.legend(handles=handles, fontsize=18)

fig.savefig(figures_path / 'svm_hyperplane_alignment_sted_pi.pdf', dpi=400)
fig.savefig(figures_path / 'svm_hyperplane_alignment_sted_pi.png', dpi=400)

## Betti Curve STED

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import svm

df_tmp = pd.read_csv(Path(vectorization_sted, 'vectorizations_all_sted_clip_minmax_gaussian2c_minmax.csv'))
selection = [x for x in df_tmp.columns if 'labels' not in x and 'bc' in x and int(x.split('_')[2]) == 3]
Xt = df_tmp[selection].values
Xt = MinMaxScaler().fit_transform(Xt)
y = df_tmp['labels'].copy()

# Assume X and y are already defined
pca = PCA(n_components=3)
X = pca.fit_transform(Xt)

varexplained = np.sum(pca.explained_variance_ratio_)
print(varexplained)

# Train SVM with a linear kernel
model = svm.SVC(kernel='linear')
clf = model.fit(X, y)

# Extract hyperplane parameters
w = clf.coef_[0]  # Normal vector to the hyperplane
b = clf.intercept_[0]

# Compute two orthonormal basis vectors that span the hyperplane
w_norm = w / np.linalg.norm(w)  # Normalize w
U, _, _ = np.linalg.svd(np.eye(3) - np.outer(w_norm, w_norm))  # Find orthonormal basis
basis_1, basis_2 = U[:, 1], U[:, 2]  # Two vectors spanning the hyperplane

# Project points onto the hyperplane
distances = (X @ w + b) / np.linalg.norm(w)  # Signed distances
X_hyper = X - np.outer(distances, w_norm)  # Projected points

# Transform hyperplane points into 2D
X_2D = np.column_stack([X_hyper @ basis_1, X_hyper @ basis_2])
Xorg_2D = np.column_stack([X @ basis_1, X @ basis_2])

# Rotate to align with the diagonal (45-degree rotation)
theta = np.pi
rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
X_2D_rotated = X_2D @ rotation_matrix
Xorg_2D_rotated = Xorg_2D @ rotation_matrix

y_unq = np.unique(X_2D_rotated[:,1])
assert np.all(np.diff(y_unq) < 1e-14)
y_unq = np.mean(y_unq)


red = (1.0, 0.0, 0.0)
blue = (0.0, 0.0, 1.0)
colors = [red if yx == 0 or yx == 'ES' else blue for yx in y]

# Scatter plot in 2D
plt.figure(figsize=(8, 6), dpi=400)
plt.hlines(0, np.min(Xorg_2D_rotated[:, 0]), np.max(Xorg_2D_rotated[:, 0]), color='black', linestyle='--')
plt.scatter(Xorg_2D_rotated[:, 0], Xorg_2D_rotated[:, 1]-y_unq,
            color=colors, marker='x', label="Original Points")

# Labels
plt.xlabel("Projected Dimension 1")
plt.ylabel("Projected Dimension 2")
plt.title("2D Projection: Hyperplane Aligned to Diagonal")
plt.legend()
plt.grid()

In [ ]:
import matplotlib.lines as mlines

plt.rcParams['font.family'] = 'Arial'

# Scatter plot in 2D
fig, ax = plt.subplots(figsize=(10,10), dpi=400)
ax.hlines(0, np.min(X_2D_rotated[:, 0]), np.max(X_2D_rotated[:, 0]), color='black')

ypredict = model.predict(X)
idx_right = ypredict == y
idx_wrong = ypredict != y

# ax.scatter(Xorg_2D_rotated[idx_right, 0], Xorg_2D_rotated[idx_right, 1]-y_unq,
#            c=y[idx_right],
#            cmap="coolwarm",
#            marker='.',
#            label="Correct Points")
# ax.scatter(Xorg_2D_rotated[idx_wrong, 0], Xorg_2D_rotated[idx_wrong, 1]-y_unq,
#            c=y[idx_wrong],
#            cmap="coolwarm",
#            marker='x',
#            label="Incorrect Points")

red = (1.0, 0.0, 0.0)
blue = (0.0, 0.0, 1.0)
colors = [red if yx == 0 or yx == 'ES' else blue for yx in y]

idx_sort = np.argsort(Xorg_2D_rotated[:, 0])
Xplot = Xorg_2D_rotated[idx_sort, :]
yplot = y[idx_sort]
idx_right = idx_right[idx_sort]
idx_wrong = idx_wrong[idx_sort]

colors_right = [red if yx == 0 or yx == 'ES' else blue for yx in yplot[idx_right]]
colors_wrong = [red if yx == 0 or yx == 'ES' else blue for yx in yplot[idx_wrong]]

ax.scatter(Xplot[idx_right, 0], Xplot[idx_right, 1]-y_unq,
           c=colors_right,
           marker='.',
           s=120, #edgecolors='black', #alpha=0.8,
           label="Correct Points")
ax.scatter(Xplot[idx_wrong, 0], Xplot[idx_wrong, 1]-y_unq,
           c=colors_wrong,
           marker='x',
           s=100, #edgecolors='black',
           lw=2,
           label="Incorrect Points")

# we have two outliers; we remove them
# sortedxvals = np.sort(Xorg_2D_rotated[:, 0])
# lastx_incluster = sortedxvals[np.argmax(np.diff(sortedxvals))]

# idx_inside = np.where(Xorg_2D_rotated[:, 0] < lastx_incluster)[0]
# ax.set_xlim(sortedxvals[0]-0.5, lastx_incluster+0.5)
# ax.set_ylim(np.min(Xorg_2D_rotated[idx_inside, 1])-y_unq-0.1,
#             np.max(Xorg_2D_rotated[idx_inside, 1])-y_unq+0.1)

# Labels
plt.xlabel("projected dimension 1", fontsize=18)
plt.ylabel("projected dimension 2", fontsize=18)
plt.title("2D Projection of PCA of Persistence Betti Curves\nAccuracy = {:.1f}% for {:d} samples".format(np.mean(idx_right)*100, len(idx_right)),
          fontsize=20)
# Add two points to the legend

handles, labels = ax.get_legend_handles_labels()
colors = plt.cm.coolwarm([0,300])

# handles.append(plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=red, markersize=6, label='ES'))
# handles.append(plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=blue, markersize=6, label='NS'))

handles = [mlines.Line2D([], [], marker='s', color='w', markerfacecolor=red, markersize=6, label='ES'),
           mlines.Line2D([], [], marker='s', color='w', markerfacecolor=blue, markersize=6, label='NS'),
           mlines.Line2D([], [], marker='x', color='black', markersize=6, linestyle=None, lw=0, label='wrong'),
           mlines.Line2D([], [], marker='o', color='w', markerfacecolor='black', markersize=6, label='correct')
           ]
ax.legend(handles=handles, fontsize=18)

fig.savefig(figures_path / 'svm_hyperplane_alignment_sted_bc.pdf', dpi=400)
fig.savefig(figures_path / 'svm_hyperplane_alignment_sted_bc.png', dpi=400)

## Airyscan 

In [ ]:


df_tmp = pd.read_csv(Path(vectorization_airyscan, 'vectorizations_all_airyscan_clip_minmax_gaussian2c_minmax.csv'))
selection = [x for x in df_tmp.columns if 'labels' not in x and 'bc' in x and int(x.split('_')[2]) == 3]
Xt = df_tmp[selection].values
Xt = MinMaxScaler().fit_transform(Xt)
y = df_tmp['labels'].copy()

# Assume X and y are already defined
pca = PCA(n_components=3)
X = pca.fit_transform(Xt)

print(np.sum(pca.explained_variance_ratio_))

svm = SVC(kernel='linear')
svm.fit(X, y)
print(svm.score(X, y))

# Get the separating hyperplane
w = svm.coef_[0]
a = -w[0] / w[2]

xx = np.linspace(min(X[:, 0]), max(X[:, 0]))
yy = np.linspace(min(X[:, 1]), max(X[:, 1]))
YY, XX = np.meshgrid(yy, xx)
# YY, XX = np.meshgrid(np.unique(X[:,1]), np.unique(X[:,0]))

# Plot the separating hyperplane
if abs(w[2]) > 1e-10:
    ZZ = (-w[0] * XX - w[1] * YY - svm.intercept_[0]) / w[2]
else:
    ZZ = np.full_like(XX, np.mean(X[:, 2]))

# ax.plot_surface(XX, YY, ZZ, color='black', alpha=0.9)
mask = (ZZ >= np.min(X[:,2])) & (ZZ <= np.max(X[:,2]))
XX, YY, ZZ = XX[mask], YY[mask], ZZ[mask]

fig = plt.figure(figsize=(8, 8), dpi=300)
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(XX.ravel(), YY.ravel(), ZZ.ravel(), color='black', alpha=0.4)

# plot the points
idx_right = np.where(svm.predict(X) == y)[0]
idx_wrong = np.where(svm.predict(X) != y)[0]

colors_right = np.array([red if yx == 0 or yx == 'ES' else blue for yx in y[idx_right]])
colors_wrong = np.array([red if yx == 0 or yx == 'ES' else blue for yx in y[idx_wrong]])

red = (1.0, 0.0, 0.0)
blue = (0.0, 0.0, 1.0)
colors = np.array([red if yx == 0 or yx == 'ES' else blue for yx in y])

# Scatter plot with color based on y values
sc = ax.scatter(X[idx_right, 0], X[idx_right, 1], X[idx_right, 2], 
                c=colors_right, marker='o', alpha=0.7)
sc = ax.scatter(X[idx_wrong, 0], X[idx_wrong, 1], X[idx_wrong, 2], 
                c=colors_wrong, marker='x', alpha=0.7)

ax.set_box_aspect([1,1,1])
ax.margins(0)

# ax.azim = 1.87
ax.azim = 10
ax.elev = 20
ax.dist = 1

ax.set_title('Principal Component Analysis for Airyscan (Betti Curve)', y=0.98)# \nexplained variance = {:.2f}'.format(np.sum(pca.explained_variance_ratio_)))
fig.savefig(figures_path / 'pca_3d_svm_hyperplane_airyscan_clip_minmax_gaussian2c_minmax_bc.pdf', dpi=300)

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Create figure
fig = go.Figure()

# Scatter plot of data points
fig.add_trace(go.Scatter3d(
    x=X[idx_right, 0],
    y=X[idx_right, 1],
    z=X[idx_right, 2],
    mode='markers',
    marker=dict(color=colors[idx_right], size=5),
    showlegend=True
))
fig.add_trace(go.Scatter3d(
    x=X[idx_wrong, 0],
    y=X[idx_wrong, 1],
    z=X[idx_wrong, 2],
    mode='markers',
    marker=dict(color=colors[idx_wrong], size=3, symbol='x')
))
fig.update_layout(
    width=1000,  # Increase width
    height=800,  # Increase height
)

# Generate hyperplane grid
xx = np.linspace(min(X[:, 0]), max(X[:, 0]), 20)
yy = np.linspace(min(X[:, 1]), max(X[:, 1]), 20)
YY, XX = np.meshgrid(yy, xx)

if abs(w[2]) > 1e-6:
    ZZ = (-w[0] * XX - w[1] * YY - svm.intercept_[0]) / w[2]
else:
    ZZ = np.full_like(XX, np.mean(X[:, 2]))

# Add hyperplane as a surface
fig.add_trace(go.Surface(x=XX, y=YY, z=ZZ, colorscale='Greys', opacity=0.5, showscale=False))

# Update layout for interactivity
fig.update_layout(
    scene=dict(
        xaxis_title="PC1",
        yaxis_title="PC2",
        zaxis_title="PC3"
    ),
    # title="Interactive 3D SVM Hyperplane",
    showlegend=False,
    scene_camera=dict(
        eye=dict(x=-1.87, y=-0.88, z=-0.64)
    )
)
# Extract camera settings from a Plotly figure
camera = fig.layout.scene.camera

azim = camera.eye.x  # Azimuth-like value
elev = camera.eye.y  # Elevation-like value
dist = camera.eye.z  # Distance from the plot

print(f"Azimuth: {azim}, Elevation: {elev}, Distance: {dist}")

fig.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import svm

df_tmp = pd.read_csv(Path(vectorization_airyscan, 'vectorizations_all_airyscan_clip_minmax_gaussian2c_minmax.csv'))
selection = [x for x in df_tmp.columns if 'labels' not in x and 'bc' in x and int(x.split('_')[2]) == 3]
Xt = df_tmp[selection].values
Xt = MinMaxScaler().fit_transform(Xt)
y = df_tmp['labels'].copy()

# Assume X and y are already defined
pca = PCA(n_components=3)
X = pca.fit_transform(Xt)

varexplained = np.sum(pca.explained_variance_ratio_)
print(varexplained)

# Train SVM with a linear kernel
model = svm.SVC(kernel='linear')
clf = model.fit(X, y)

# Extract hyperplane parameters
w = clf.coef_[0]  # Normal vector to the hyperplane
b = clf.intercept_[0]

# Compute two orthonormal basis vectors that span the hyperplane
w_norm = w / np.linalg.norm(w)  # Normalize w
U, _, _ = np.linalg.svd(np.eye(3) - np.outer(w_norm, w_norm))  # Find orthonormal basis
basis_1, basis_2 = U[:, 1], U[:, 2]  # Two vectors spanning the hyperplane

# Project points onto the hyperplane
distances = (X @ w + b) / np.linalg.norm(w)  # Signed distances
X_hyper = X - np.outer(distances, w_norm)  # Projected points

# Transform hyperplane points into 2D
X_2D = np.column_stack([X_hyper @ basis_1, X_hyper @ basis_2])
Xorg_2D = np.column_stack([X @ basis_1, X @ basis_2])

# Rotate to align with the diagonal (45-degree rotation)
theta = np.pi
rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
X_2D_rotated = X_2D @ rotation_matrix
Xorg_2D_rotated = Xorg_2D @ rotation_matrix

y_unq = np.unique(X_2D_rotated[:,1])
assert np.all(np.diff(y_unq) < 1e-14)
y_unq = np.mean(y_unq)


red = (1.0, 0.0, 0.0)
blue = (0.0, 0.0, 1.0)
colors = [red if yx == 0 or yx == 'ES' else blue for yx in y]

# Scatter plot in 2D
plt.figure(figsize=(8, 6), dpi=400)
plt.hlines(0, np.min(Xorg_2D_rotated[:, 0]), np.max(Xorg_2D_rotated[:, 0]), color='black', linestyle='--')
plt.scatter(Xorg_2D_rotated[:, 0], Xorg_2D_rotated[:, 1]-y_unq,
            color=colors, marker='x', label="Original Points")

# Labels
plt.xlabel("Projected Dimension 1")
plt.ylabel("Projected Dimension 2")
plt.title("2D Projection: Hyperplane Aligned to Diagonal")
plt.legend()
plt.grid()

In [ ]:
import matplotlib.lines as mlines

plt.rcParams['font.family'] = 'Arial'

# Scatter plot in 2D
fig, ax = plt.subplots(figsize=(10,10), dpi=400)
ax.hlines(0, np.min(X_2D_rotated[:, 0]), np.max(X_2D_rotated[:, 0]), color='black')

ypredict = model.predict(X)
idx_right = ypredict == y
idx_wrong = ypredict != y

# ax.scatter(Xorg_2D_rotated[idx_right, 0], Xorg_2D_rotated[idx_right, 1]-y_unq,
#            c=y[idx_right],
#            cmap="coolwarm",
#            marker='.',
#            label="Correct Points")
# ax.scatter(Xorg_2D_rotated[idx_wrong, 0], Xorg_2D_rotated[idx_wrong, 1]-y_unq,
#            c=y[idx_wrong],
#            cmap="coolwarm",
#            marker='x',
#            label="Incorrect Points")

red = (1.0, 0.0, 0.0)
blue = (0.0, 0.0, 1.0)
colors = [red if yx == 0 or yx == 'ES' else blue for yx in y]

idx_sort = np.argsort(Xorg_2D_rotated[:, 0])
Xplot = Xorg_2D_rotated[idx_sort, :]
yplot = y[idx_sort]
idx_right = idx_right[idx_sort]
idx_wrong = idx_wrong[idx_sort]

colors_right = [red if yx == 0 or yx == 'ES' else blue for yx in yplot[idx_right]]
colors_wrong = [red if yx == 0 or yx == 'ES' else blue for yx in yplot[idx_wrong]]

ax.scatter(Xplot[idx_right, 0], Xplot[idx_right, 1]-y_unq,
           c=colors_right,
           marker='.',
           s=120, #edgecolors='black', #alpha=0.8,
           label="Correct Points")
ax.scatter(Xplot[idx_wrong, 0], Xplot[idx_wrong, 1]-y_unq,
           c=colors_wrong,
           marker='x',
           s=100, #edgecolors='black',
           lw=2,
           label="Incorrect Points")

# we have two outliers; we remove them
# sortedxvals = np.sort(Xorg_2D_rotated[:, 0])
# lastx_incluster = sortedxvals[np.argmax(np.diff(sortedxvals))]

# idx_inside = np.where(Xorg_2D_rotated[:, 0] < lastx_incluster)[0]
# ax.set_xlim(sortedxvals[0]-0.5, lastx_incluster+0.5)
# ax.set_ylim(np.min(Xorg_2D_rotated[idx_inside, 1])-y_unq-0.1,
#             np.max(Xorg_2D_rotated[idx_inside, 1])-y_unq+0.1)

# Labels
plt.xlabel("dim 1", fontsize=18)
plt.ylabel("dim 2", fontsize=18)
plt.title("Airyscan: 2D Projection of PCA of Persistence Betti Curves\nAccuracy = {:.1f}% for {:d} samples".format(np.mean(idx_right)*100, len(idx_right)),
          fontsize=20)
# Add two points to the legend

handles, labels = ax.get_legend_handles_labels()
colors = plt.cm.coolwarm([0,300])

# handles.append(plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=red, markersize=6, label='ES'))
# handles.append(plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=blue, markersize=6, label='NS'))

handles = [mlines.Line2D([], [], marker='s', color='w', markerfacecolor=red, markersize=6, label='ES'),
           mlines.Line2D([], [], marker='s', color='w', markerfacecolor=blue, markersize=6, label='NS'),
           mlines.Line2D([], [], marker='x', color='black', markersize=6, linestyle=None, lw=0, label='wrong'),
           mlines.Line2D([], [], marker='o', color='w', markerfacecolor='black', markersize=6, label='correct')
           ]
ax.legend(handles=handles, fontsize=18)

fig.savefig(figures_path / 'svm_hyperplane_alignment_airyscan_bc.pdf', dpi=400)
fig.savefig(figures_path / 'svm_hyperplane_alignment_airyscan_bc.png', dpi=400)